#### Script di estrazione tweets tramite cursore, con creazione dei dataframe legati ai retweet e ai tweet retwittati

In [7]:
#Importazione librerie
import tweepy
import pandas as pd
from tqdm import tqdm

In [8]:
#accessi Twitter
consumer_key= ""
consumer_secret= ""
access_token= ""
access_token_secret= ""

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

In [10]:

#raccolta per hashtag tramite cursore Tweepy, estraendo le informazioni dai tweet utili alle analisi
hashtags=["#NoGreenPass","#vaccinatevi","@CasaPoundItalia","@matteosalvinimi","@EnricoLetta"]
list_tweet=[]
list_originals=[]
list_rt_id=[]
list_retweetted=[]


for hashtag in hashtags:
  next_cursor=None
  n=5000
  pbar = tqdm(total=n)
  while n>0:
    for tweet in tweepy.Cursor(api.search_tweets,q=hashtag,lang="it", tweet_mode='extended').items(n): #,cursor=next_cursor

      #gestione dei retweet con try-except
      try:
        list_originals.append(tweet.retweeted_status.id) #creo lista dei retweets id 
        list_rt_id.append(tweet.retweeted_status.user.id)

        #lista tweets che sono stati retweettati
        try: 
          rt_text=tweet.retweeted_status.full_text
        except AttributeError:
          rt_text=tweet.retweeted_status.text

        list_retweetted.append([tweet.retweeted_status.created_at, tweet.retweeted_status.id,tweet.retweeted_status.user.id, rt_text, 
                         tweet.retweeted_status.favorite_count, tweet.retweeted_status.retweet_count, 
                         tweet.retweeted_status.user.screen_name,hashtag]) 
      except:
        list_originals.append(0)
        list_rt_id.append(0) 

      try: 
        tweet_text=tweet.full_text
      except AttributeError:
        tweet_text=tweet.text

      list_tweet.append([tweet.created_at, tweet.id,tweet.user.id, tweet_text, 
                         tweet.favorite_count, tweet.retweet_count, 
                         tweet.user.location,tweet.user.screen_name,
                         hasattr(tweet, "retweeted_status"),
                         tweet.in_reply_to_screen_name,hashtag])
      n -=1
      #next_cursor=tweet
      pbar.update(n=1)


  0%|          | 0/5000 [38:13:14<?, ?it/s]





















100%|██████████| 5000/5000 [38:56:23<00:00, 28.04s/it]































































































































































Rate limit reached. Sleeping for: 784



























































































































































  7%|▋         | 373/5000 [00:16<03:20, 23.12it/s]Rate limit reached. Sleeping for: 794

100%|██████████| 5000/5000 [30:19<00:00,  2.75it/s]  
Rate limit reached. Sleeping for: 787
Rate limit reached. Sleeping for: 791
100%|██████████| 5000/5000 [29:57<00:00,  2.78it/s]  
Rate limit reached. Sleeping for: 787
Rate limit reached. Sleeping for: 787


In [205]:
#creo df dei tweet
tweets = pd.DataFrame(list_tweet, 
                      columns=['date','id','user_id','text','like','n_rt','location','author',
                               "retweet","reply_name",'hashtags'])


tweets["id_retweet"]=list_originals
tweets["user_id_retweet"]=list_rt_id


#creo df dei retweet
retweetted = pd.DataFrame(list_retweetted, 
                      columns=['date','id','author_id','text','like','n_rt','author','hashtags'])    


tweets.head()

,date,id,user_id,text,like,n_rt,location,author,retweet,reply_name,hashtags,id_retweet,user_id_retweet
0,2021-12-30 10:55:21+00:00,1476507191133843457,442083142,RT @Lorenzo62752880: La logica alla base del G...,0,122,Milano,mspini2,True,None,#NoGreenPass,1476217276525387776,1364298062919794688
1,2021-12-30 10:53:56+00:00,1476506837730140161,305255857,RT @GianniMagini: Questo scemo urlava col mega...,0,7,,Rosyfree74,True,None,#NoGreenPass,1476483050703929346,977383349235961857
2,2021-12-30 10:53:41+00:00,1476506773590880258,950098070771036160,"RT @GandalfRevn: ""La prima volta nella storia ...",0,354,,GrandeTornado1,True,None,#NoGreenPass,1444582094265634816,1438546161955229703
3,2021-12-30 10:52:07+00:00,1476506377614942208,229160327,RT @Lorenzo62752880: Crisanti al Corriere:\n\n...,0,149,altrove🇮🇹🇮🇹🇮🇹🇮🇹🇮🇹,marcoranieri72,True,None,#NoGreenPass,1476289519402078209,1364298062919794688
4,2021-12-30 10:51:48+00:00,1476506297356926982,229160327,RT @Lorenzo62752880: @borghi_claudio Ovviament...,0,68,altrove🇮🇹🇮🇹🇮🇹🇮🇹🇮🇹,marcoranieri72,True,None,#NoGreenPass,1476297527129432073,1364298062919794688


In [207]:
#creo un subset dei tweet integrato con informazioni dei retweet tra cui hashtag
tweets_2=tweets.merge(retweetted[['id','text']],left_on='id_retweet',right_on='id',how='left')[['date','id_x','user_id','text_x','author','retweet','id_retweet','hashtags','user_id_retweet','text_y']].rename(columns={'text_y':'text_complete'})

In [208]:
tweets_2=tweets_2[tweets_2.text_complete.notnull()].drop_duplicates().reset_index(drop=True)

In [212]:
#arricchimento df tweets classificando dei gruppi in base all'hashtag
weigth_column=[]

for index,row in tweets_2.iterrows():
    if row['hashtags']=="#NoGreenPass":
        weigth_column.append(0)
    if row['hashtags']=="#vaccinatevi":
        weigth_column.append(1)
    if row['hashtags']=="@CasaPoundItalia":
        weigth_column.append(0)
    if row['hashtags']=="@matteosalvinimi":
        weigth_column.append(0)
    if row['hashtags']=="@EnricoLetta":
        weigth_column.append(1)

tweets_2['gruppo']=weigth_column

In [214]:
weigth_column_r=[]

for index,row in retweetted.iterrows():
    if row['hashtags']=="#NoGreenPass":
        weigth_column_r.append(0)
    if row['hashtags']=="#vaccinatevi":
        weigth_column_r.append(1)
    if row['hashtags']=="@CasaPoundItalia":
        weigth_column_r.append(0)
    if row['hashtags']=="@matteosalvinimi":
        weigth_column_r.append(0)
    if row['hashtags']=="@EnricoLetta":
        weigth_column_r.append(1)

retweetted['gruppo']=weigth_column_r


In [210]:
retweetted=retweetted.drop_duplicates(['id','author_id']).reset_index(drop=True)

In [ ]:
tweets_2.to_csv('Tweet_definitivo.csv')
retweetted.to_csv('Retweetted_definitivo.csv')

In [211]:
#selezione dei tweet più retwettati 
retweetted.sort_values("id",ascending=False)

,date,id,author_id,text,like,n_rt,author,hashtags
1337,2021-12-30 12:48:21+00:00,1476535628531707908,1388915991686664196,@ALFO100897 @EnricoLetta Alfo ma il Super Gree...,2,2,PeterWo31671308,@EnricoLetta
1340,2021-12-30 12:47:39+00:00,1476535452182138881,1020303953974759424,"@marilenagasbar1 @EnricoLetta Solo ""qualcosa"",...",1,1,ALFO100897,@EnricoLetta
1339,2021-12-30 12:45:14+00:00,1476534844435972101,843385951929950208,@EnricoLetta @il_cappellini Su amazon le trovi...,2,1,theStewieee,@EnricoLetta
1341,2021-12-30 12:32:32+00:00,1476531648388907008,513144334,"@EnricoLetta @il_cappellini Quindi, Enrico, il...",3,1,RenatoSouvarine,@EnricoLetta
977,2021-12-30 12:25:46+00:00,1476529946877177859,938345883279724544,@MauroV1968 @FranceskoNew @ASampierdarena @Car...,3,2,ScuolaNoCovid,@EnricoLetta
...,...,...,...,...,...,...,...,...
701,2020-11-01 10:22:08+00:00,1322846377475481601,270839361,"🔴‼️Stamane a #Lampedusa: oltre 100, e si parla...",2996,863,matteosalvinimi,@matteosalvinimi
1163,2019-09-01 12:12:01+00:00,1168134413621846017,419622371,Un gran segno di #speranza! Che bella scelta! ...,1021,204,EnricoLetta,@EnricoLetta
928,2019-08-13 12:34:25+00:00,1161254678409175041,270839361,Simpatico questo vignettista di molti giornali...,4598,976,matteosalvinimi,@matteosalvinimi
668,2019-05-31 18:35:06+00:00,1134528748521480194,831646859378774016,Se #Assange muore saranno responsabili oltre a...,584,275,Wikileaks_Ita,@matteosalvinimi


#### Creazione input Grephi tramite scrittura file .csv

In [216]:
rt_tweets=tweets_2[tweets_2.retweet==True][['user_id','user_id_retweet','author','gruppo','hashtags']]

In [217]:
tweets_ids=rt_tweets[['user_id','author','gruppo','hashtags']]
retwitted_ids=retweetted[['author_id','author','gruppo','hashtags']].rename(columns={'author_id':'user_id'})
rt_tweets_ids=pd.concat([tweets_ids,retwitted_ids]).sort_values(by=['hashtags'],ascending=False).groupby(['user_id','author']).head(1).reset_index(drop=True)

In [218]:
Nodes_df=pd.DataFrame({'Id':rt_tweets_ids.user_id,'Label':rt_tweets_ids.author,'Interval':None,'Hashtag':rt_tweets_ids.hashtags,'Gruppo':rt_tweets_ids.gruppo})

In [219]:
Edges_df=pd.DataFrame({'Source':rt_tweets.user_id,'Target':rt_tweets.user_id_retweet,'Type':'Directed','Kind':None,'Id':range(1, len(rt_tweets)+1),'Label':None,'Interval':None,'Weight':1.0})

In [220]:
Nodes_df.to_csv('Nodes_df6.csv', index=False)
Edges_df.to_csv('Edges_df6.csv', index=False)